# DIWASP-Python: End-to-End Wave Analysis Demonstration

This notebook demonstrates the complete workflow for directional wave spectrum analysis using DIWASP-Python.

We'll cover:
1. Creating synthetic wave data with varying conditions
2. Analyzing data with different sensor configurations
3. Comparing different estimation methods
4. Visualizing results

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from diwasp import diwasp, makespec, make_wave_data

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Steady Sea State Analysis

First, let's analyze a steady sea state with a single dominant wave direction.

In [ ]:
# Create an idealized JONSWAP spectrum
freqs = np.linspace(0.05, 0.5, 50)
dirs = np.linspace(0, 360, 181, endpoint=False)

ideal_spec = makespec(
    freqs=freqs,
    dirs=dirs,
    spreading=75,           # Directional spreading parameter
    frequency_hz=0.1,       # Peak frequency (10s waves)
    direction_deg=45,       # Peak direction (from NE)
    gamma=3.3,              # JONSWAP peak enhancement factor
)

print(f"Ideal spectrum created:")
print(f"  Hsig: {ideal_spec.hsig:.2f} m")
print(f"  Tp: {ideal_spec.tp:.2f} s")
print(f"  Peak direction: {ideal_spec.dp:.1f} deg")

In [ ]:
# Generate synthetic sensor data (PUV configuration)
fs = 2.0  # 2 Hz sampling
duration = 3600  # 1 hour

# Sensor layout: pressure at 0.5m, velocities at 1.0m above seabed
layout = np.array([
    [0, 0, 0],      # x positions
    [0, 0, 0],      # y positions  
    [0.5, 1.0, 1.0] # z positions
])

data = make_wave_data(
    spec=ideal_spec,
    layout=layout,
    datatypes=['pres', 'velx', 'vely'],
    depth=20.0,
    fs=fs,
    duration=duration,
)

print(f"Generated {data.shape[0]} samples ({duration}s at {fs}Hz)")
print(f"Sensors: pressure, u-velocity, v-velocity")

In [ ]:
# Create DataFrame for analysis
n_samples = int(duration * fs)
time = pd.date_range('2024-01-01', periods=n_samples, freq=f'{int(1000/fs)}ms')

df = pd.DataFrame({
    'pressure': data[:, 0],
    'u_velocity': data[:, 1],
    'v_velocity': data[:, 2],
}, index=time)

# Quick look at the data
fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

axes[0].plot(df.index[:600], df['pressure'][:600])
axes[0].set_ylabel('Pressure (m)')
axes[0].set_title('First 5 minutes of sensor data')
axes[0].grid(True)

axes[1].plot(df.index[:600], df['u_velocity'][:600])
axes[1].set_ylabel('U velocity (m/s)')
axes[1].grid(True)

axes[2].plot(df.index[:600], df['v_velocity'][:600])
axes[2].set_ylabel('V velocity (m/s)')
axes[2].set_xlabel('Time')
axes[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Run DIWASP analysis with 30-minute windows, 15-minute overlap
result = diwasp(
    df,
    sensor_mapping={'pressure': 'pres', 'u_velocity': 'velx', 'v_velocity': 'vely'},
    window_length=1800,      # 30 minutes
    window_overlap=900,       # 15 minutes overlap
    depth=20.0,
    z={'pressure': 0.5, 'u_velocity': 1.0, 'v_velocity': 1.0},
    method='imlm',
    verbose=1,
)

print(f"\nAnalysis complete!")
print(f"Output shape: {result['efth'].shape}")
print(f"Number of time windows: {len(result.time)}")

In [ ]:
# Plot time series of wave parameters
fig, axes = plt.subplots(4, 1, figsize=(12, 10), sharex=True)

axes[0].plot(result.time, result.hsig, 'o-', label='Estimated')
axes[0].axhline(ideal_spec.hsig, color='r', linestyle='--', label='Ideal')
axes[0].set_ylabel('Hsig (m)')
axes[0].set_title('Wave Parameters Over Time')
axes[0].legend()
axes[0].grid(True)

axes[1].plot(result.time, result.tp, 'o-', label='Estimated')
axes[1].axhline(ideal_spec.tp, color='r', linestyle='--', label='Ideal')
axes[1].set_ylabel('Tp (s)')
axes[1].legend()
axes[1].grid(True)

axes[2].plot(result.time, result.dp, 'o-', label='Estimated')
axes[2].axhline(ideal_spec.dp, color='r', linestyle='--', label='Ideal')
axes[2].set_ylabel('Peak Dir (deg)')
axes[2].legend()
axes[2].grid(True)

axes[3].plot(result.time, result.spread, 'o-', label='Estimated')
axes[3].axhline(ideal_spec.spread, color='r', linestyle='--', label='Ideal')
axes[3].set_ylabel('Spread (deg)')
axes[3].set_xlabel('Time')
axes[3].legend()
axes[3].grid(True)

plt.tight_layout()
plt.show()

# Print summary statistics
print(f"\nSummary Statistics:")
print(f"  Hsig: {result.hsig.mean().values:.2f} ± {result.hsig.std().values:.2f} m (ideal: {ideal_spec.hsig:.2f} m)")
print(f"  Tp: {result.tp.mean().values:.2f} ± {result.tp.std().values:.2f} s (ideal: {ideal_spec.tp:.2f} s)")
print(f"  Peak Dir: {result.dp.mean().values:.1f} ± {result.dp.std().values:.1f} deg (ideal: {ideal_spec.dp:.1f} deg)")

In [ ]:
# Plot 2D spectrum for first window
fig = plt.figure(figsize=(14, 5))
gs = GridSpec(1, 3, figure=fig)

# Ideal spectrum
ax1 = fig.add_subplot(gs[0, 0])
c1 = ax1.contourf(ideal_spec.dirs, ideal_spec.freqs, ideal_spec.S, levels=20, cmap='viridis')
ax1.set_xlabel('Direction (deg)')
ax1.set_ylabel('Frequency (Hz)')
ax1.set_title('Ideal Spectrum')
plt.colorbar(c1, ax=ax1, label='Energy (m²/Hz/deg)')

# Estimated spectrum (first window)
ax2 = fig.add_subplot(gs[0, 1])
c2 = ax2.contourf(result.dir, result.freq, result.efth[0], levels=20, cmap='viridis')
ax2.set_xlabel('Direction (deg)')
ax2.set_ylabel('Frequency (Hz)')
ax2.set_title('Estimated Spectrum (Window 1)')
plt.colorbar(c2, ax=ax2, label='Energy (m²/Hz/deg)')

# Frequency spectra comparison
ax3 = fig.add_subplot(gs[0, 2])
ideal_freq_spec = ideal_spec.S.sum(axis=1) * (dirs[1] - dirs[0])
est_freq_spec = result.efth[0].sum(axis=1) * (result.dir[1] - result.dir[0]).values
ax3.plot(ideal_spec.freqs, ideal_freq_spec, 'r-', linewidth=2, label='Ideal')
ax3.plot(result.freq, est_freq_spec, 'b-', linewidth=2, label='Estimated')
ax3.set_xlabel('Frequency (Hz)')
ax3.set_ylabel('Energy (m²/Hz)')
ax3.set_title('Frequency Spectrum')
ax3.legend()
ax3.grid(True)

plt.tight_layout()
plt.show()

## 2. Varying Sea State Analysis

Now let's simulate a changing sea state with varying wave height, period, and direction.

In [ ]:
# Create time-varying wave conditions
duration_per_segment = 1800  # 30 minutes per segment
segments = [
    {'hsig': 1.5, 'tp': 8, 'dir': 30, 'label': 'Calm'},
    {'hsig': 2.0, 'tp': 10, 'dir': 45, 'label': 'Building'},
    {'hsig': 2.5, 'tp': 12, 'dir': 60, 'label': 'Peak'},
    {'hsig': 2.0, 'tp': 10, 'dir': 45, 'label': 'Declining'},
]

all_data = []
segment_times = []

for i, seg in enumerate(segments):
    # Create spectrum for this segment
    spec = makespec(
        freqs=np.linspace(0.05, 0.5, 50),
        dirs=np.linspace(0, 360, 181, endpoint=False),
        spreading=75,
        frequency_hz=1.0/seg['tp'],
        direction_deg=seg['dir'],
        gamma=3.3,
    )
    
    # Generate data
    segment_data = make_wave_data(
        spec=spec,
        layout=layout,
        datatypes=['pres', 'velx', 'vely'],
        depth=20.0,
        fs=fs,
        duration=duration_per_segment,
    )
    
    all_data.append(segment_data)
    segment_times.append((i * duration_per_segment, (i + 1) * duration_per_segment))
    
    print(f"Segment {i+1} ({seg['label']}): Hsig={seg['hsig']}m, Tp={seg['tp']}s, Dir={seg['dir']}°")

# Concatenate all segments
data_varying = np.vstack(all_data)
total_duration = len(segments) * duration_per_segment
n_samples = int(total_duration * fs)

time_varying = pd.date_range('2024-01-01', periods=n_samples, freq=f'{int(1000/fs)}ms')
df_varying = pd.DataFrame({
    'p': data_varying[:, 0],
    'u': data_varying[:, 1],
    'v': data_varying[:, 2],
}, index=time_varying)

In [ ]:
# Analyze varying sea state
result_varying = diwasp(
    df_varying,
    sensor_mapping={'p': 'pres', 'u': 'velx', 'v': 'vely'},
    window_length=900,   # 15 minutes
    window_overlap=450,  # 7.5 minutes overlap
    depth=20.0,
    z={'p': 0.5, 'u': 1.0, 'v': 1.0},
    method='imlm',
    verbose=1,
)

In [ ]:
# Plot time series showing variation
fig, axes = plt.subplots(3, 1, figsize=(14, 9), sharex=True)

# Add shaded regions for each segment
colors = ['lightblue', 'lightgreen', 'lightyellow', 'lightcoral']
for i, (seg, (t_start, t_end)) in enumerate(zip(segments, segment_times)):
    for ax in axes:
        ax.axvspan(time_varying[int(t_start*fs)], time_varying[int(t_end*fs)-1], 
                   alpha=0.3, color=colors[i], label=seg['label'] if ax == axes[0] else '')

axes[0].plot(result_varying.time, result_varying.hsig, 'ko-', linewidth=2, markersize=6)
axes[0].set_ylabel('Hsig (m)', fontsize=12)
axes[0].set_title('Varying Sea State Analysis', fontsize=14, fontweight='bold')
axes[0].legend(loc='upper left')
axes[0].grid(True, alpha=0.3)

axes[1].plot(result_varying.time, result_varying.tp, 'ko-', linewidth=2, markersize=6)
axes[1].set_ylabel('Tp (s)', fontsize=12)
axes[1].grid(True, alpha=0.3)

axes[2].plot(result_varying.time, result_varying.dp, 'ko-', linewidth=2, markersize=6)
axes[2].set_ylabel('Peak Direction (deg)', fontsize=12)
axes[2].set_xlabel('Time', fontsize=12)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nDetected variation:")
print(f"  Hsig range: {result_varying.hsig.min().values:.2f} - {result_varying.hsig.max().values:.2f} m")
print(f"  Tp range: {result_varying.tp.min().values:.1f} - {result_varying.tp.max().values:.1f} s")
print(f"  Direction range: {result_varying.dp.min().values:.1f} - {result_varying.dp.max().values:.1f} deg")

## 3. Comparison of Estimation Methods

Let's compare different estimation methods on the same data.

In [ ]:
# Create test data
test_spec = makespec(
    freqs=np.linspace(0.05, 0.5, 50),
    dirs=np.linspace(0, 360, 181, endpoint=False),
    spreading=75,
    frequency_hz=0.125,  # 8 second waves
    direction_deg=90,    # From East
    gamma=3.3,
)

test_data = make_wave_data(
    spec=test_spec,
    layout=layout,
    datatypes=['pres', 'velx', 'vely'],
    depth=20.0,
    fs=2.0,
    duration=1800,
)

time_test = pd.date_range('2024-01-01', periods=test_data.shape[0], freq='500ms')
df_test = pd.DataFrame({
    'p': test_data[:, 0],
    'u': test_data[:, 1],
    'v': test_data[:, 2],
}, index=time_test)

print(f"Test spectrum: Hsig={test_spec.hsig:.2f}m, Tp={test_spec.tp:.1f}s, Dir={test_spec.dp:.0f}°")

In [ ]:
# Run analysis with different methods
methods = ['dftm', 'emlm', 'imlm', 'emep', 'bdm']
results_methods = {}

for method in methods:
    print(f"\nRunning {method.upper()}...")
    result = diwasp(
        df_test,
        sensor_mapping={'p': 'pres', 'u': 'velx', 'v': 'vely'},
        window_length=1800,
        window_overlap=0,
        depth=20.0,
        z=1.0,
        method=method,
        verbose=0,
    )
    results_methods[method] = result
    print(f"  Hsig: {result.hsig.values[0]:.2f} m")
    print(f"  Tp: {result.tp.values[0]:.2f} s")
    print(f"  Peak Dir: {result.dp.values[0]:.1f} deg")

In [ ]:
# Compare methods
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

# Plot ideal spectrum in first panel
c0 = axes[0].contourf(test_spec.dirs, test_spec.freqs, test_spec.S, levels=20, cmap='viridis')
axes[0].set_xlabel('Direction (deg)')
axes[0].set_ylabel('Frequency (Hz)')
axes[0].set_title('Ideal Spectrum', fontweight='bold')
plt.colorbar(c0, ax=axes[0], label='Energy (m²/Hz/deg)')

# Plot each method
for i, method in enumerate(methods, start=1):
    result = results_methods[method]
    c = axes[i].contourf(result.dir, result.freq, result.efth[0], levels=20, cmap='viridis')
    axes[i].set_xlabel('Direction (deg)')
    axes[i].set_ylabel('Frequency (Hz)')
    axes[i].set_title(f'{method.upper()} Method', fontweight='bold')
    plt.colorbar(c, ax=axes[i], label='Energy (m²/Hz/deg)')

plt.tight_layout()
plt.show()

# Summary table
print("\n" + "="*70)
print(f"{'Method':<10} {'Hsig (m)':<12} {'Tp (s)':<12} {'Peak Dir (deg)':<15} {'Error Hsig':<12}")
print("="*70)
print(f"{'IDEAL':<10} {test_spec.hsig:<12.2f} {test_spec.tp:<12.2f} {test_spec.dp:<15.1f} {'-':<12}")
print("-"*70)
for method in methods:
    result = results_methods[method]
    hsig_err = abs(result.hsig.values[0] - test_spec.hsig) / test_spec.hsig * 100
    print(f"{method.upper():<10} {result.hsig.values[0]:<12.2f} {result.tp.values[0]:<12.2f} "
          f"{result.dp.values[0]:<15.1f} {hsig_err:<12.1f}%")
print("="*70)

## 4. Pressure Gauge Array Analysis

Demonstrate analysis with a triangular array of pressure sensors.

In [ ]:
# Create spectrum
array_spec = makespec(
    freqs=np.linspace(0.05, 0.5, 50),
    dirs=np.linspace(0, 360, 181, endpoint=False),
    spreading=75,
    frequency_hz=0.1,
    direction_deg=270,  # From West
    gamma=3.3,
)

# Triangular array layout (5m spacing)
array_layout = np.array([
    [0, 5, -5],   # x positions
    [0, 5, 5],    # y positions
    [0, 0, 0],    # z positions (on seabed)
])

array_data = make_wave_data(
    spec=array_spec,
    layout=array_layout,
    datatypes=['pres', 'pres', 'pres'],
    depth=15.0,
    fs=2.0,
    duration=3600,
)

time_array = pd.date_range('2024-01-01', periods=array_data.shape[0], freq='500ms')
df_array = pd.DataFrame({
    'p1': array_data[:, 0],
    'p2': array_data[:, 1],
    'p3': array_data[:, 2],
}, index=time_array)

print(f"Pressure array configuration:")
print(f"  Sensor 1: x={array_layout[0,0]:.1f}m, y={array_layout[1,0]:.1f}m")
print(f"  Sensor 2: x={array_layout[0,1]:.1f}m, y={array_layout[1,1]:.1f}m")
print(f"  Sensor 3: x={array_layout[0,2]:.1f}m, y={array_layout[1,2]:.1f}m")
print(f"\nIdeal: Hsig={array_spec.hsig:.2f}m, Tp={array_spec.tp:.1f}s, Dir={array_spec.dp:.0f}°")

In [ ]:
# Analyze array data
result_array = diwasp(
    df_array,
    sensor_mapping={'p1': 'pres', 'p2': 'pres', 'p3': 'pres'},
    window_length=1800,
    window_overlap=900,
    depth=15.0,
    x={'p1': 0, 'p2': 5, 'p3': -5},
    y={'p1': 0, 'p2': 5, 'p3': 5},
    z=0.0,
    method='emep',
    verbose=1,
)

In [ ]:
# Visualize array results
fig = plt.figure(figsize=(14, 10))
gs = GridSpec(2, 2, figure=fig)

# Array layout
ax1 = fig.add_subplot(gs[0, 0])
ax1.scatter(array_layout[0, :], array_layout[1, :], s=200, c='red', marker='o', edgecolors='black', linewidths=2)
for i in range(3):
    ax1.annotate(f'P{i+1}', (array_layout[0, i], array_layout[1, i]), 
                xytext=(5, 5), textcoords='offset points', fontsize=12, fontweight='bold')
ax1.set_xlabel('X (m)', fontsize=12)
ax1.set_ylabel('Y (m)', fontsize=12)
ax1.set_title('Pressure Gauge Array Layout', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.axis('equal')

# Time series
ax2 = fig.add_subplot(gs[0, 1])
ax2.plot(result_array.time, result_array.hsig, 'o-', linewidth=2, markersize=8)
ax2.axhline(array_spec.hsig, color='r', linestyle='--', linewidth=2, label='Ideal')
ax2.set_xlabel('Time', fontsize=12)
ax2.set_ylabel('Hsig (m)', fontsize=12)
ax2.set_title('Significant Wave Height', fontsize=13, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 2D spectrum
ax3 = fig.add_subplot(gs[1, :])
c = ax3.contourf(result_array.dir, result_array.freq, result_array.efth[0], levels=20, cmap='viridis')
ax3.set_xlabel('Direction (deg)', fontsize=12)
ax3.set_ylabel('Frequency (Hz)', fontsize=12)
ax3.set_title('Estimated Directional Spectrum (First Window)', fontsize=13, fontweight='bold')
plt.colorbar(c, ax=ax3, label='Energy (m²/Hz/deg)')

plt.tight_layout()
plt.show()

print(f"\nArray Analysis Results:")
print(f"  Mean Hsig: {result_array.hsig.mean().values:.2f} m (ideal: {array_spec.hsig:.2f} m)")
print(f"  Mean Tp: {result_array.tp.mean().values:.2f} s (ideal: {array_spec.tp:.2f} s)")
print(f"  Mean Peak Dir: {result_array.dp.mean().values:.1f} deg (ideal: {array_spec.dp:.1f} deg)")

## Summary

This notebook demonstrated:

1. **Steady sea state analysis** - Creating synthetic data and analyzing with the DIWASP wrapper
2. **Varying conditions** - Tracking changes in wave parameters over time
3. **Method comparison** - Comparing different estimation algorithms (DFTM, EMLM, IMLM, EMEP, BDM)
4. **Pressure array** - Using multiple sensors in different locations

Key features of DIWASP-Python:
- Accepts pandas DataFrame or xarray Dataset
- Automatic windowing with overlap
- Multiple estimation methods
- Flexible sensor configurations
- wavespectra-compatible output